In [ ]:
#Runtime subvoxel tests
import time
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from disimpy import gradients, simulations, substrates, utils

n_walkers = int(1e4)
diffusivity = 2e-9  # SI units (m^2/s)
gradient = np.zeros((1, int(1e4), 3))
dt = (1e-6)*(1e-6)/(6*diffusivity)

def calc_time(subvoxel_division, time_table, time_table1):
    # Load an example triangular mesh
    mesh_path = os.path.join(
        os.path.dirname(simulations.__file__), "tests", "neuron-model.pkl"
    )
    with open(mesh_path, "rb") as f:
        example_mesh = pickle.load(f)
    faces = example_mesh["faces"]
    vertices = example_mesh["vertices"]

    # Create a substrate object
    init_time = time.time()
    substrate = substrates.mesh(
        vertices, faces, padding=np.zeros(3), periodic=True, init_pos="uniform", n_sv=subvoxel_division
    )
    fin_time = time.time()
    total_time = fin_time-init_time
    time_table.append(total_time)

    # Run simulation
    traj_file = "example_traj.txt"

    init_time1 = time.time()
    signals = simulations.simulation(
        n_walkers=n_walkers,
        diffusivity=diffusivity,
        gradient=gradient,
        dt=dt,
        substrate=substrate,
        traj=traj_file,
    )
    fin_time1 = time.time()
    total_time1 = fin_time1-init_time1
    time_table1.append(total_time1)

def main_time():    
    time_table=[]
    time_table1=[]
    division=np.array([1, 1, 1])
    calc_time(division, time_table, time_table1)
    for x in range (5,51,5):
        division=np.array([x, x, x])
        calc_time(division, time_table, time_table1)

    return time_table, time_table1

time_table_def, time_table_def1 = main_time()
print(time_table_def,time_table_def1)

fig, ax = plt.subplots()
plt.xlim(0,50)
plt.ylim(0,5)
ax.plot([1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50], time_table_def)
plt.xlabel('Number of subvoxels in each axis')
plt.ylabel('Substrate creation time (s)')

fig, ax = plt.subplots()
plt.xlim(0,50)
plt.ylim(0,10)
ax.plot([1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50], time_table_def1)
plt.xlabel('Number of subvoxels in each axis')
plt.ylabel('Simulation runtime (s)')

In [ ]:
#Permeable triangles
@numba.jit(nopython=True)
def _cuda_crossing(r0, step, d, normal, epsilon):
    """Calculate intersection and update r0. Epsilon is the amount by
    which the new position differs from the surface.
    Parameters
    ----------
    r0 : numba.cuda.cudadrv.devicearray.DeviceNDArray
    step : numba.cuda.cudadrv.devicearray.DeviceNDArray
    d : float
    normal : numba.cuda.cudadrv.devicearray.DeviceNDArray
    semiaxes : numba.cuda.cudadrv.devicearray.DeviceNDArray
    epsilon : float
    Returns
    -------
    float
    """
    intersection = cuda.local.array(3, numba.float64)
    v = cuda.local.array(3, numba.float64)
    for i in range(3):
        intersection[i] = r0[i] + d * step[i]
        v[i] = intersection[i] - r0[i]
    dp = _cuda_dot_product(v, normal)
    if dp < 0: # Make sure the normal vector points to the other side of the triangle
        for i in range(3):
            normal[i] *= -1
        dp = _cuda_dot_product(v, normal)
    for i in range(3): # Move walker slightly away from the surface
        r0[i] = intersection[i] + epsilon * normal[i]
    return

In [ ]:
@cuda.jit()
def _cuda_step_mesh(
    positions,
    g_x,
    g_y,
    g_z,
    phases,
    rng_states,
    t,
    step_l,
    dt,
    vertices,
    faces,
    xs,
    ys,
    zs,
    subvoxel_indices,
    triangle_indices,
    iter_exc,
    max_iter,
    n_sv,
    epsilon,
):
    """Kernel function for diffusion restricted by a triangular mesh."""

    thread_id = cuda.grid(1)
    if thread_id >= positions.shape[0]:
        return

    # Allocate memory
    step = cuda.local.array(3, numba.float64)
    lls = cuda.local.array(3, numba.int64)
    uls = cuda.local.array(3, numba.int64)
    triangle = cuda.local.array((3, 3), numba.float64)
    normal = cuda.local.array(3, numba.float64)
    shifts = cuda.local.array(3, numba.float64)
    temp_r0 = cuda.local.array(3, numba.float64)

    # Get position and generate step
    r0 = positions[thread_id, :]
    _cuda_random_step(step, rng_states, thread_id)

    # Check for intersection, reflect step, and repeat until no intersection
    check_intersection = True
    iter_idx = 0
    while check_intersection and step_l > 0 and iter_idx < max_iter:
        iter_idx += 1
        min_d = math.inf

        # Find the relevant subvoxels for this step
        lls[0] = _ll_subvoxel_overlap_periodic(xs, r0[0], r0[0] + step[0] * step_l)
        lls[1] = _ll_subvoxel_overlap_periodic(ys, r0[1], r0[1] + step[1] * step_l)
        lls[2] = _ll_subvoxel_overlap_periodic(zs, r0[2], r0[2] + step[2] * step_l)
        uls[0] = _ul_subvoxel_overlap_periodic(xs, r0[0], r0[0] + step[0] * step_l)
        uls[1] = _ul_subvoxel_overlap_periodic(ys, r0[1], r0[1] + step[1] * step_l)
        uls[2] = _ul_subvoxel_overlap_periodic(zs, r0[2], r0[2] + step[2] * step_l)

        # Loop over subvoxels and fnd the closest triangle
        for x in range(lls[0], uls[0]):

            # Check if subvoxel is outside the simulated voxel
            if x < 0 or x > len(xs) - 2:
                shift_n = math.floor(x / (len(xs) - 1))
                x -= shift_n * (len(xs) - 1)
                shifts[0] = shift_n * xs[-1]
            else:
                shifts[0] = 0

            for y in range(lls[1], uls[1]):

                # Check if subvoxel is outside the simulated voxel
                if y < 0 or y > len(ys) - 2:
                    shift_n = math.floor(y / (len(ys) - 1))
                    y -= shift_n * (len(ys) - 1)
                    shifts[1] = shift_n * ys[-1]
                else:
                    shifts[1] = 0

                for z in range(lls[2], uls[2]):

                    # Check if subvoxel is outside the simulated voxel
                    if z < 0 or z > len(zs) - 2:
                        shift_n = math.floor(z / (len(zs) - 1))
                        z -= shift_n * (len(zs) - 1)
                        shifts[2] = shift_n * zs[-1]
                    else:
                        shifts[2] = 0

                    # Find the corresponding subvoxel in the simulated voxel
                    sv = int(x * n_sv[1] * n_sv[2] + y * n_sv[2] + z)

                    for i in range(3):  # Move walker to the simulated voxel
                        temp_r0[i] = r0[i] - shifts[i]

                    # Loop over the triangles in this subvoxel
                    for i in range(subvoxel_indices[sv, 0], subvoxel_indices[sv, 1]):
                        _cuda_get_triangle(
                            triangle_indices[i], vertices, faces, triangle
                        )
                        d = _cuda_ray_triangle_intersection_check(
                            triangle, temp_r0, step
                        )
                        if d > 0 and d < min_d:
                            closest_triangle_index = triangle_indices[i]
                            min_d = d

        # Check if step intersects with the closest triangle
        rand_numb = xoroshiro128p_uniform_float64(rng_states, thread_id)
        if min_d < step_l and rand_numb > perm_prob:
                _cuda_get_triangle(closest_triangle_index, vertices, faces, triangle)
                _cuda_triangle_normal(triangle, normal)
                _cuda_reflection(r0, step, min_d, normal, epsilon)
                step_l -= min_d
        elif min_d > step_l:
                check_intersection = False
        elif rand_numb < perm_prob:
            _cuda_get_triangle(closest_triangle_index, vertices, faces, triangle)
            _cuda_triangle_normal(triangle, normal)
            _cuda_crossing(r0, step, min_d, normal, epsilon)
            step_l -= min_d

    if iter_idx >= max_iter:
        iter_exc[thread_id] = True
    for i in range(3):
        positions[thread_id, i] = r0[i] + step[i] * step_l
    for m in range(g_x.shape[0]):
        phases[m, thread_id] += (
            GAMMA
            * dt
            * (
                (g_x[m, t] * positions[thread_id, 0])
                + (g_y[m, t] * positions[thread_id, 1])
                + (g_z[m, t] * positions[thread_id, 2])
            )
        )
    return

In [ ]:
#Mesh space subdivision method
from disimpy import simulations, substrates
import os, meshio
import numpy as np
import numba

mesh_path = os.path.join(
    os.path.dirname(simulations.__file__), "tests", "neuron-model.stl"
)
mesh = meshio.read(mesh_path)
vertices = mesh.points.astype(np.float32)
faces = mesh.cells[0].data

padding = np.zeros(3)
shift = -np.min(vertices, axis=0)
vertices = vertices + shift
voxel_size = np.max(vertices, axis=0) + padding
n_sv = np.array([10,10,10])

@numba.jit()
def str_to_int(s):
    final_index, result = len(s) - 1, 0
    for i,v in enumerate(s):
        result += (ord(v) - 48) * (10 ** (final_index - i))
    return result

@numba.jit()
def _mesh_space_subdivision_vnumb(vertices, faces, voxel_size, n_sv):
    # Define the subvoxel boundaries
    xs = np.linspace(0, voxel_size[0], n_sv[0] + 1)
    ys = np.linspace(0, voxel_size[1], n_sv[1] + 1)
    zs = np.linspace(0, voxel_size[2], n_sv[2] + 1)
    #relevant_triangles = ["" for _ in range(np.product(n_sv))] 
    prod = n_sv[0]*n_sv[1]*n_sv[2]
    relevant_triangles = ["" for _ in range(prod)] 
    len_subvoxel = ["" for _ in range(prod)] 

    # Loop over the triangles
    for i, idx in enumerate(faces):
        triangle = vertices[idx]
        subvoxels = substrates._box_subvoxel_overlap(substrates._triangle_aabb(triangle), xs, ys, zs)
        for x in range(subvoxels[0, 0], subvoxels[0, 1]):
            for y in range(subvoxels[1, 0], subvoxels[1, 1]):
                for z in range(subvoxels[2, 0], subvoxels[2, 1]):
                    box = np.array(
                        [[xs[x], ys[y], zs[z]], [xs[x + 1], ys[y + 1], zs[z + 1]]]
                    )
                    if _triangle_box_overlap(triangle, box):
                        subvoxel = x * n_sv[1] * n_sv[2] + y * n_sv[2] + z
                        digits = len(str(i))
                        relevant_triangles[subvoxel] += (str(i)+"-")
                        len_subvoxel[subvoxel] += (str(digits))

    # Make the final arrays
    triangle_indices = []
    subvoxel_indices = np.zeros((len(relevant_triangles), 2))
    last_idx = 0
    first_idx = 0
    for i, l in enumerate(relevant_triangles):
        position = 0
        if l == '':
            subvoxel_indices[i,0] = first_idx
            subvoxel_indices[i,1] = first_idx
            continue
        end = False
        ctr = 0 
        subvoxel_indices[i, 0] = first_idx
        while (end == False):
            triangle= str_to_int(l[position:position+str_to_int(len_subvoxel[i][ctr])])
            triangle_indices.append(triangle)
            ctr += 1
            if ctr == len(len_subvoxel[i]):
                subvoxel_indices[i, 1] = first_idx+ctr
                end = True
            else:
                position += str_to_int(len_subvoxel[i][ctr-1])+1
        first_idx += ctr

    return xs, ys, zs, triangle_indices, subvoxel_indices

In [ ]:
@numba.jit()
def _triangle_box_overlap(triangle, box):
    # Move the box and triangle so that the box's centre is at the origin
    c = np.array([np.mean(box[:, i]) for i in range(3)])
    h = np.abs(box[1] - box[0]) / 2
    v = triangle - c
    e = np.eye(3)

    # Test the triangle AABB against the box
    triangle_aabb = _triangle_aabb(v)
    for i in range(3):
        if (triangle_aabb[1][i] < box[0][i] or triangle_aabb[0][i] > box[1][i]):
            return False

    # Test the plane in which the triangle is against the box
    f = np.array(
        [
            [v[1, 0] - v[0, 0], v[1, 1] - v[0, 1], v[1, 2] - v[0, 2]],
            [v[2, 0] - v[1, 0], v[2, 1] - v[1, 1], v[2, 2] - v[1, 2]],
            [v[0, 0] - v[2, 0], v[0, 1] - v[2, 1], v[0, 2] - v[2, 2]],
        ]
    )
    normal = _cross_product(f[0], f[1])
    corners = np.array(
        [
            [h[0], h[1], h[2]],
            [-h[0], -h[1], -h[2]],
            [-h[0], h[1], h[2]],
            [h[0], -h[1], -h[2]],
            [h[0], -h[1], h[2]],
            [-h[0], h[1], -h[2]],
            [h[0], h[1], -h[2]],
            [-h[0], -h[1], h[2]],
        ]
    )
    in_plane = False
    behind = np.zeros(8, dtype=numba.boolean)
    for i, point in enumerate(corners):
        dp = _dot_product(normal, v[0] - point)
        if dp == 0:
            in_plane = True
        else:
            behind[i] = _dot_product(normal, v[0] - point) > 0
    if not in_plane and (np.all(behind) or np.all(~behind)):
        return False

    # Test the triangle against the box
    p = np.zeros(3)
    for i in range(3):
        for j in range(3):
            a = _cross_product(e[i], f[j])
            r = _dot_product(h, np.abs(a))
            for k in range(3):
                p[k] = _dot_product(a, v[k])
            if np.min(p) > r or np.max(p) < -r:
                return False
    return True

In [ ]:
import numba, numpy as np
from numba import cuda
from disimpy import simulations

@cuda.jit(device=True)
def _cuda_ray_triangle_intersection_check(triangle, r0, step):
    A = triangle[0]
    B = triangle[1]
    C = triangle[2]
    T = cuda.local.array(3, numba.float64)
    E_1 = cuda.local.array(3, numba.float64)
    E_2 = cuda.local.array(3, numba.float64)
    P = cuda.local.array(3, numba.float64)
    Q = cuda.local.array(3, numba.float64)

    for i in range(3):
        T[i] = r0[i] - A[i]
        E_1[i] = B[i] - A[i]
        E_2[i] = C[i] - A[i]
    simulations._cuda_cross_product(step, E_2, P)
    simulations._cuda_cross_product(T, E_1, Q)
    det = simulations._cuda_dot_product(P, E_1)
    if det != 0:
        t = 1 / det * simulations._cuda_dot_product(Q, E_2)
        u = 1 / det * simulations._cuda_dot_product(P, T)
        v = 1 / det * simulations._cuda_dot_product(Q, step)
        if u >= 0 and u <= 1 and v >= 0 and v <= 1 and u + v <= 1:
            return t
        else:
            return np.nan
    else:
        return np.nan
import numpy.testing as npt

def test__cuda_ray_triangle_intersection_check():
    @cuda.jit()
    def test_kernel(ds, triangle, r0s, steps):
        thread_id = cuda.grid(1)
        if thread_id >= ds.shape[0]:
            return
        ds[thread_id, :] = simulations._cuda_ray_triangle_intersection_check(
            triangle, r0s[thread_id, :], steps[thread_id, :]
        )
        return

    triangle = np.array([[2.0, 0, 0], [0, 2.0, 0], [0.0, 0, 0]])
    r0s = np.array(
        [
            [0.1, 0.1, 1.0],
            [0.1, 0.1, 1.0],
            [0.1, 0.1, 1.0],
            [0.1, 0.1, 1.0],
            [10, 10, 0],
        ]
    )
    steps = np.array(
        [[0, 0, -1.0], [0, 0, 1], [0, 0, -0.1], [1.0, 1.0, 0], [0, 0, 1.0]]
    )
    ds = np.zeros((5, 1))
    stream = cuda.stream()
    test_kernel[1, 256, stream](ds, triangle, r0s, steps)
    stream.synchronize()
    npt.assert_almost_equal(ds, np.array([[1, -1, 10, np.nan, np.nan]]).T)
    return